In [11]:
# from numba import jit
import numpy as np
import matplotlib.pyplot as plt
from types import SimpleNamespace
import time
from copy import deepcopy


In [12]:
# Configure the system's initial conditions
def conf_ini(N, boxsize, temper):
    L = int(
        np.sqrt(N)
    )  # Define a dimensao linear da rede quadrada  (Atencao!!! A raiz quadrada de N é suposta inteira)

    x = np.zeros(N)
    y = np.zeros(N)

    vx = np.zeros(N)
    vy = np.zeros(N)

    cont = 0
    for i in range(L):
        for j in range(L):
            # distribui as partículas numa rede quadrada centrada
            x[cont] = (i + 0.5) - L / 2
            y[cont] = (j + 0.5) - L / 2

            # Escolhe a velocidade uniformemente num circulo unitario
            phi = np.random.uniform(0, 2 * np.pi)
            vx[cont] = np.cos(phi)
            vy[cont] = np.sin(phi)

            cont += 1

    # normaliza as posições para dentro da caixa
    x = x / L * boxsize
    y = y / L * boxsize

    # normaliza as velocidades de acordo com o teorema da equiparticao da energia
    prov = np.sqrt((2.0 - 2.0 / N) * temper)

    vx = vx * prov
    vy = vy * prov

    # Zera o momento total das partículas
    prov = np.sum(vx)
    vx = vx - prov / N
    prov = np.sum(vy)
    vy = vy - prov / N

    return SimpleNamespace(**{"x": x, "y": y, "vx": vx, "vy": vy})


# Add random perturbation to the positions
def perturbate(state, part_size, amount, boxsize):
    new_state = deepcopy(state)
    for i in range(N):
        phi = np.random.uniform(0, 2 * np.pi)
        new_state.x[i] = calc_coord(
            new_state.x[i] + np.cos(phi) * part_size * amount, boxsize
        )
        new_state.y[i] = calc_coord(
            new_state.y[i] + np.sin(phi) * part_size * amount, boxsize
        )

    return new_state


# Calculate the distance between particles i and j
def calc_dist(i, j, state, boxsize):
    x = state.x
    y = state.y
    xij = x[i] - x[j]  # Distancia entre as partículas
    yij = y[i] - y[j]

    xij = calc_coord(xij, boxsize)  # aplica condicoes de contorno periódicas
    yij = calc_coord(yij, boxsize)

    r2 = xij**2 + yij**2

    return r2, xij, yij


# Create the verlet list
def verlet_list(
    state, rv, boxsize
):  # Cria uma lista de Verlet para definir os vizinhos
    x = state.x
    y = state.y

    N = len(x)  # numero de particulas

    nviz = np.zeros(N, np.int64)  # declara os arrays
    viz = np.empty(0, np.int64)

    cont = 0  # contador do numero de vizinhos (indice do array viz)

    for i in range(N):  # loop sobre todas as partículas
        for j in range(i + 1, N):  # loop sobre todos os possiveis vizinhos

            r2, *_ = calc_dist(i, j, state, boxsize)

            if r2 < rv:
                cont += 1
                nviz[i] = cont
                viz = np.append(viz, [j])

    return nviz, viz


# Calculate the potential at a distance r
def lennard_jones_pot(r, depth, part_size):
    return 4 * depth * ((part_size / r) ** (12) - (part_size / r ** (6)))


# Calculate the force at a distance r
def lennard_jones_force(r, depth, part_size):
    return 48 * depth * (part_size ** (12) / r ** (14) - part_size ** (6) / r ** (8))


# Apply periodic boundary conditions
def calc_coord(dist, boxsize):
    return dist - boxsize * np.rint(dist / boxsize)


# Velocity-verlet position step
def position_step(rt, vt, at, dt, boxsize):
    r_step = rt + dt * vt + dt**2 * at / 2

    return calc_coord(r_step, boxsize)


# Velocity-verlet velocity step
def velocity_step(vt, at, a_step, dt):
    return vt + dt * (at + a_step) / 2


def time_step(state, dt, boxsize):
    nviz, viz = verlet_list(state, rv, boxsize)

    state.ax = np.zeros(N)
    state.ay = np.zeros(N)

    state.pot = np.zeros(N)

    for i in range(N):
        nviz_i = nviz[i]
        viz_i = viz[nviz[i - 1] : nviz_i]

        axi = state.ax[i]
        ayi = state.ay[i]

        poti = state.pot[i]

        for j in viz_i:
            r2, xij, yij = calc_dist(i, j, state, boxsize)
            if i == 0:
                print(r2)
            # print(state.ay[0])
            if r2 < 0.77 * part_size:
                raise Exception("Position threshold reached")

            apply_potential = r2 < rcut

            if apply_potential:
                potential = lennard_jones_pot(np.sqrt(r2), depth, part_size)
                force = lennard_jones_force(np.sqrt(r2), depth, part_size)

                r_ang = np.arccos(xij**2 / r2)

                axi += (
                    0
                    if xij == 0
                    else (xij / np.abs(xij)) * force * np.cos(r_ang) / mass
                )
                ayi += (
                    0
                    if yij == 0
                    else (yij / np.abs(yij)) * force * np.sin(r_ang) / mass
                )

                poti += potential / mass

            state.pot[j] += poti
            state.ax[j] -= axi
            state.ay[j] -= ayi

        state.x[i] = position_step(state.x[i], state.vx[i], state.ax[i], dt, boxsize)
        state.y[i] = position_step(state.y[i], state.vy[i], state.ay[i], dt, boxsize)

        state.vx[i] = velocity_step(state.vx[i], state.ax[i], axi, dt)
        state.vy[i] = velocity_step(state.vy[i], state.ay[i], ayi, dt)

        state.ax[i] += axi
        state.ay[i] += ayi

        state.pot[i] += poti


def calc_pos_deviation(ref_state, pert_state):
    sum = 0
    for i in range(N):
        pert_pos = np.sqrt(pert_state.x[i] ** 2 + pert_state.y[i] ** 2)
        ref_pos = np.sqrt(ref_state.x[i] ** 2 + ref_state.y[i] ** 2)

        sum += (pert_pos - ref_pos) ** 2

    avg_deviation = sum / N

    return avg_deviation


def calc_energy(state):
    kin_energy = 0
    pot_energy = 0

    for i in range(N):
        v2 = state.vx[i] ** 2 + state.vy[i] ** 2
        kin_energy += mass * v2 / 2

        pot_energy += state.pot[i]

    return kin_energy, pot_energy


def plot_dev_over_time(dev_over_time):
    fig, ax = plt.subplots(1, 1)

    plt.yscale("log")
    ax.plot(dev_over_time)


In [13]:
L = 10
N = L * L
mass = 1

# densidade
rho = 0.6
# dimensao linear da caixa
boxsize = np.sqrt(N / rho)

temper = 1.05

# Raio de corte Lennard-Jones
rcut = 3
rcut = rcut**2

# Raio para lista de verlet
rv = 2
rv = rv**2

# Parameters of the lennard-jones potential
# (https://www.researchgate.net/figure/Lennard-Jones-parameters-for-argon-and-carbon-atoms_tbl1_348928432)
depth = 10.3 * 10 ** (-3)
part_size = 3.405 * 10 ** (-10) * boxsize

dt = 0.005


In [14]:
ref_state = conf_ini(N, boxsize, temper)
# nviz, viz = verlet_list(ref_state, rv, boxsize)
# plt.scatter(ref_state.x, ref_state.y)
# plt.show()

# for i in range(N):
#     nviz_i = nviz[i]
#     viz_i = viz[nviz[i - 1] : nviz_i]

#     print(i, viz_i)


In [15]:
pert_state = perturbate(ref_state, part_size, 10 ** (-3), boxsize)
iterations = 200

i = 0
st = time.time()

dev_over_time = np.array([])
kin_energy_over_time = np.array([])
pot_energy_over_time = np.array([])

while i < iterations:
    i += 1
    time_step(ref_state, dt, boxsize)
    time_step(pert_state, dt, boxsize)

    deviation = calc_pos_deviation(ref_state, pert_state)
    dev_over_time = np.append(dev_over_time, deviation)

    kin_energy, pot_energy = calc_energy(ref_state)
    kin_energy_over_time = np.append(kin_energy_over_time, kin_energy)
    pot_energy_over_time = np.append(pot_energy_over_time, pot_energy)

print("Number of iterations:", i)

# et = time.time()
# get the execution time
# elapsed_time = et - st
# print('Execution time:', elapsed_time, 'seconds')


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'calc_dist':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-12-9be2c66dbecb>", line 90:[0m
[1mdef verlet_list(
    <source elided>

[1m            r2, xij, yij = calc_dist(i, j, state, boxsize)
[0m            [1m^[0m[0m
[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'types.SimpleNamespace'>[0m


In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(kin_energy_over_time)

print(pot_energy_over_time)
